# **GIS × Python Tutorial Session10 ~ DSMの作成とDCHMの作成 ~**
---
この記事は「GIS × Python Tutorial」の関連記事です。

<br><br><br>



## **はじめに**
---


<br><br><br>

 - DSMの作成
    - 点群データの読み込み
    - DSMの計算
    - 可視化（DSMとDTMを並べる）
 - DCHMの作成
    - DSMからDTMを引く方法
    - PDALで点群データからDTMを引き、DSMを作成する方法
    - 可視化（DCHM - DSM - DTMを並べる）

In [1]:
import pdal
import rasterio